In [ ]:
from modules import dataloaders, schedulers, model, train
from torch import nn, optim

In [ ]:
print(torch.cuda.is_available(), torch.backends.cudnn.enabled)
cuda_flag = torch.cuda.is_available()
device = torch.device('cuda') if cuda_flag else torch.device('cpu')

In [ ]:
dl, _ = dataloaders.get_cifar10_data_loaders(data_dir='./data/cifar10', batch_size=100)
rgb_ave = dataloaders.channel_avg(dl)

In [ ]:
def record_run(train_dl:torch.utils.data.DataLoader,
               test_dl:torch.utils.data.DataLoader,
               scheduler:optim.lr_scheduler._LRScheduler,
               result_path:str):
    'TODO:docstring'
    net = model.Cifar10Net().to(device)
    init_weights(net)
    crit = nn.CrossEntropyLoss()
    opt = optim.SGD(net.parameters(), lr=0.0001, momentum=0.9, weight_decay=0.004)
    recorder = {
        'iteration' : [],
        'trn_loss' : [],
        'lr' : [],
        'val_loss' : [],
        'val_acc' : []}
    obe = partial(on_batch_end, recorder, test_dl, net, crit)
    train_run(net, train_dl, crit, opt, scheduler, 24000, obe, device)
    return pd.DataFrame(recorder).to_csv(result_path, index=False)

In [ ]:
tfms = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=rgb_ave, std=[1,1,1])])
train_dl, test_dl = get_cifar10_data_loaders(data_dir='./data/cifar10',
                                             batch_size=100,
                                             num_workers=8,
                                             pin_memory=cuda_flag,
                                             train_transform=tfms,
                                             test_transform=tfms)

t_sched = TriangularScheduler(step_size=2000, min_lr=0.001, max_lr=0.006, optimizer=opt)
record_run(train_dl, test_dl, t_sched, './results/triangular.csv')